In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.app import PermutationSolver


In [3]:
n = 12
config = {
            'n_permutations_length': n,
            
            # Random walks params
            'random_walks_type': 'non-backtracking-beam',
            'n_random_walk_length': int(n * (n-1) / 2),
            'n_random_walks_to_generate': 10000,
            'n_random_walks_steps_back_to_ban': 8,
            
            # Neural Net params
            'model_type': 'MLP',
            'list_layers_sizes': [2**9],
            'n_epochs': 30,
            'batch_size': 1024,
            'lr': 0.001,
            
            # DQN training
            'n_epochs_dqn': 300,
            'flag_dqn_round': False,
            'n_random_walks_to_generate_dqn': 1000,
            'verbose_loc': 5,
            'lr_dqn': 0.0005,
            
            # Beam search
            'beam_search_torch': True,
            'beam_search_Fironov': False,
            'beam_width': 1,
            'n_steps_limit': 4 * n**2,
            'alpha_previous_cost_accumulation': 0,
            'beam_search_models_or_heuristics': 'model_torch',
            'ban_p0_p1_transposition_if_p0_lt_p1_ie_already_sorted': False,
            'n_beam_search_steps_back_to_ban': 32,
            
            # What to solve
            'solve_random_or_longest_state': 'solve_LRX_longest',
            'verbose': 100
        }

In [4]:
# n_perm_list = [8]
# n_epoch_list = [100]
# n_epoch_dqn_list = [300]
# batch_size_list = [2**10]
# lr_list = [0.000001]
# beam_width_list = [2**0]
# list_layers_sizes = [[2**13]]

n_perm_list = [15]
n_epoch_list = [30]
n_epoch_dqn_list = [1000]
batch_size_list = [2**10]
lr_list = [0.0005]
beam_width_list = [2**0]
list_layers_sizes = [[2**11]]

In [5]:
config['mode'] = 'single_hard_hinge'
config['w_anchor'] = 1.0
config['w_hinge'] = 1.0


In [6]:
summary_df = pd.DataFrame()

for n_permutations_length in n_perm_list:
    solver = PermutationSolver(config)
    solver.config['n_permutations_length'] = n_permutations_length
    for list_layers_size in list_layers_sizes:
        solver.config['list_layers_sizes'] = list_layers_size
        for lr in lr_list:
            solver.config['lr_dqn'] = lr
            for batch_size in batch_size_list:
                solver.config['batch_size'] = batch_size
                for n_epoch in n_epoch_list:
                    solver.config['n_epochs'] = n_epoch
                    mlp_losses = solver.train_mlp()
                    # save mlp_model
                    torch.save(solver.mlp_model.state_dict(), f'models/mlp_model_{n_permutations_length}_{n_epoch}_{list_layers_sizes[0]}_{lr}_{batch_size}.pth')
                    for n_epoch_dqn in n_epoch_dqn_list:
                        solver.config['n_epochs_dqn'] = n_epoch_dqn
                        dqn_losses = solver.train_dqn()
                        # save dqn_model
                        torch.save(solver.dqn_model.state_dict(), f'models/dqn_model_{n_permutations_length}_{n_epoch_dqn}_{list_layers_sizes[0]}_{lr}_{batch_size}.pth')
                        for beam_width in beam_width_list:
                            solver.config['beam_width'] = beam_width
                            i_step, flag_found_destination = solver.test_beam_search()
                            # save summary_df
                            summary_df = pd.concat([summary_df, pd.DataFrame({'n_permutations_length': n_permutations_length, 'list_layers_sizes': list_layers_sizes, 'lr': lr, 'n_epoch': n_epoch, 'n_epoch_dqn': n_epoch_dqn, 'beam_width': beam_width, 'i_step': i_step, 'flag_found_destination': flag_found_destination, 'mlp_losses': mlp_losses[-1], 'dqn_losses': dqn_losses[-1]})])
                            summary_df.to_csv('models/summary_df.csv', index=False)


Training MLP:  33%|███▎      | 10/30 [00:11<00:23,  1.15s/it]

Epoch 10, Loss: 59.9828


Training MLP:  67%|██████▋   | 20/30 [00:22<00:10,  1.08s/it]

Epoch 20, Loss: 59.5383


Training MLP: 100%|██████████| 30/30 [00:33<00:00,  1.11s/it]

...

Epoch 990 | Loss: 0.2258 | Hinge: 0.0463 | Anchor: 0.1670 | Times - RW: 1.53s, Bellman: 0.02s, Train: 0.12s


Training DQN: 100%|█████████▉| 996/1000 [27:59<00:06,  1.68s/it]

Epoch 995 | Loss: 0.1771 | Hinge: 0.0725 | Anchor: 0.1032 | Times - RW: 1.53s, Bellman: 0.02s, Train: 0.10s


Training DQN: 100%|██████████| 1000/1000 [28:06<00:00,  1.69s/it]


Training finished in 1686.0s
i_position_X_in_list_generators: 2
Step: 1 Beam (not cumulative) min: 66.04 median: 66.18 max: 66.39
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.000, t_unique_els  0.001, t_full_step 0.013
Step: 11 Beam (not cumulative) min: 62.08 median: 63.06 max: 64.05
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.001, t_full_step 1747252702.381
Step: 21 Beam (not cumulative) min: 62.34 median: 63.08 max: 63.81
Step: 31 Beam (not cumulative) min: 62.30 median: 62.47 max: 62.63
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.000, t_unique_els  0.001, t_full_step 0.002
Step: 41 Beam (not cumulative) min: 63.96 median: 64.13 max: 64.30
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.000, t_unique_els  0.000, t_full_step 1747252702.443
Step: 51 Beam (not cumulative) min: 62.70 median: 63.17 max: 63.65
Step: 61 Beam (not cumulative) min: 62.08 median: 63.40 max: 64.71
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.000, t_unique_els  0.001

In [7]:
solver.config['n_steps_limit'] = 10000

In [9]:
solver.config['beam_width'] = 16
i_step, flag_found_destination = solver.test_beam_search()

i_position_X_in_list_generators: 2
Step: 1 Beam (not cumulative) min: 0.00 median: 0.00 max: 0.00
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.008, t_unique_els  0.005, t_full_step 0.018
Step: 11 Beam (not cumulative) min: 62.25 median: 62.95 max: 65.29
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.000, t_full_step 1747252720.598
Step: 21 Beam (not cumulative) min: 59.34 median: 61.56 max: 63.62
Step: 31 Beam (not cumulative) min: 59.77 median: 61.13 max: 63.73
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.001, t_full_step 0.003
Step: 41 Beam (not cumulative) min: 59.11 median: 61.38 max: 63.70
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.001, t_full_step 1747252720.723
Step: 51 Beam (not cumulative) min: 53.39 median: 55.93 max: 58.37
Step: 61 Beam (not cumulative) min: 45.69 median: 48.06 max: 51.00
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.001, t_full_step 0.004
Step: 71 Bea

In [ ]:
# Initialize solver
solver = PermutationSolver(config)

# Train MLP first
print("\nTraining MLP model...")
mlp_losses = solver.train_mlp()

# Train DQN using pre-trained MLP weights
print("\nTraining DQN model with pre-trained MLP weights...")
dqn_losses = solver.train_dqn()

# Test models with beam search
print("\nTesting models with beam search...")
print("\nTesting DQN model...")
dqn_result = solver.test_beam_search()

In [9]:
from collections import deque, defaultdict
import torch
import numpy as np

def bfs_build_dataset(state_destination, list_generators, device, num_of_samples=5_000_000):
    start_state = tuple(state_destination.tolist())
    queue = deque([start_state])
    visited = {start_state: 0}
    
    while queue and len(visited) < num_of_samples:
        current_state = queue.popleft()
        current_depth = visited[current_state]
        
        for gen in list_generators:
            next_state_list = [current_state[gen[i]] for i in range(len(gen))]
            next_state = tuple(next_state_list)
            
            if next_state not in visited:
                visited[next_state] = current_depth + 1
                queue.append(next_state)
    
    all_states = list(visited.keys())
    print(len(all_states))
    depths = [visited[s] for s in all_states]
    
    X = torch.tensor(all_states, dtype=torch.long)
    y = torch.tensor(depths, dtype=torch.long)

    print('X.shape:',X.shape)
    print('y.shape:',y.shape)
    
    return X.to(device), y.to(device)


def get_LRX_moves(n):
    L = np.array( list(np.arange(1,n)) + [0])
    R = np.array( [n-1] + list(np.arange(n-1)) )
    X = np.array( [1,0] + list(np.arange(2,n)) )
    return L,R,X

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
dtype_generators = torch.int64 
n = 9

L,R,X = get_LRX_moves(n)
list_generators = [L,R,X]
state_destination = torch.arange( len(list_generators[0]) , device = device, dtype =  dtype_generators  )

num_of_samples = 1_000_000
X_train, y_train = bfs_build_dataset(state_destination, list_generators, device, num_of_samples=num_of_samples)

362880
X.shape: torch.Size([362880, 9])
y.shape: torch.Size([362880])
